# 选取rawcounts矩阵以及计算RPKM进行筛选
https://github.com/reneshbedre/bioinfokit

!pip install bioinfokit

!pip install openpyxl

In [5]:
# !pip install bioinfokit
# !pip install openpyxl

In [21]:
import pandas as pd
from bioinfokit.analys import norm

In [22]:
ordered_bam_labels = [
    'Geneid', 'Length',
    'LI-rep1', 'LI-rep2',
    'mLN-rep1', 'mLN-rep2',
    'pLN-rep1', 'pLN-rep2',
    'SI-rep1', 'SI-rep2',
    'spleen-rep1', 'spleen-rep2'
]

In [23]:
fc_file = '../featureCounts/all_feature.txt'
df = pd.read_csv(fc_file, header=1, sep='\t')
df_geneid = df.iloc[:, 0:1]
# 1,2,3,4列是基因坐标，下游分析不需要，如有需要回来再找
df_counts = df.iloc[:, 5:]
df_merge = pd.concat([df_geneid, df_counts], axis=1)
df_merge.head(2)

,Geneid,Length,../bam/LI-rep1_Aligned_sort_rmdup.bam,../bam/LI-rep2_Aligned_sort_rmdup.bam,../bam/mLN-rep1_Aligned_sort_rmdup.bam,../bam/mLN-rep2_Aligned_sort_rmdup.bam,../bam/pLN-rep1_Aligned_sort_rmdup.bam,../bam/pLN-rep2_Aligned_sort_rmdup.bam,../bam/SI-rep1_Aligned_sort_rmdup.bam,../bam/SI-rep2_Aligned_sort_rmdup.bam,../bam/spleen-rep1_Aligned_sort_rmdup.bam,../bam/spleen-rep2_Aligned_sort_rmdup.bam
0,Xkr4,3634,32,40,31,33,18,17,24,26,0,29
1,Rp1,12293,90,141,124,89,56,60,142,117,0,100


In [24]:
# df = df.copy()
df_merge.columns=ordered_bam_labels
df_merge.set_index('Geneid', inplace=True)
df_merge.head(2)

,Length,LI-rep1,LI-rep2,mLN-rep1,mLN-rep2,pLN-rep1,pLN-rep2,SI-rep1,SI-rep2,spleen-rep1,spleen-rep2
Geneid,,,,,,,,,,,
Xkr4,3634,32,40,31,33,18,17,24,26,0,29
Rp1,12293,90,141,124,89,56,60,142,117,0,100


In [25]:
# now, normalize raw counts using RPKM method
# gene length must be in bp
nm = norm()
nm.rpkm(df=df_merge, gl='Length')
# get RPKM normalized dataframe
df_rpkm = nm.rpkm_norm
df_rpkm.head(2)

,LI-rep1,LI-rep2,mLN-rep1,mLN-rep2,pLN-rep1,pLN-rep2,SI-rep1,SI-rep2,spleen-rep1,spleen-rep2
Geneid,,,,,,,,,,
Xkr4,11.836652,4.751964,14.615606,19.668831,8.828403,7.084624,18.182024,19.924005,0.0,12.188404
Rp1,9.841209,4.951757,17.282393,15.681285,8.119414,7.391731,31.801409,26.504292,0.0,12.424414


In [34]:
# save rpkm
df_rpkm.to_excel('../featureCounts/all_feature_rpkm.xlsx')
# save raw_counts
df_merge.to_excel('../featureCounts/all_feature_raw_counts.xlsx')

In [36]:
# 按照 RPKM 每个 Gene 的均值大于 1 的条件过滤基因

# 或者不进行过滤
print(df_rpkm.shape[0])

flt = df_rpkm.sum(axis=1) /df_rpkm.shape[1] >= 0

26214


In [37]:
df_flt_rawcounts = df_merge[flt].copy()
df_flt_rawcounts.reset_index(inplace=True)
df_flt_rawcounts

,Geneid,Length,LI-rep1,LI-rep2,mLN-rep1,mLN-rep2,pLN-rep1,pLN-rep2,SI-rep1,SI-rep2,spleen-rep1,spleen-rep2
0,Xkr4,3634,32,40,31,33,18,17,24,26,0,29
1,Rp1,12293,90,141,124,89,56,60,142,117,0,100
2,Sox17,4095,5,9,3,2,1,3,2,2,0,4
3,Mrpl15,4201,39,40,56,36,33,53,28,44,0,39
4,Lypla1,2503,38,269,13,14,10,8,10,15,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...
26209,Vmn2r122,2740,0,0,0,0,0,0,0,0,0,0
26210,Vamp7,2578,0,0,0,0,0,0,0,0,0,0
26211,Spry3,4346,0,0,0,0,0,0,0,0,0,0
26212,Tmlhe,2257,0,0,0,0,0,0,0,0,0,0


In [38]:
df_flt_rawcounts.drop('Length', axis=1, inplace=True)

In [39]:
df_flt_rawcounts

,Geneid,LI-rep1,LI-rep2,mLN-rep1,mLN-rep2,pLN-rep1,pLN-rep2,SI-rep1,SI-rep2,spleen-rep1,spleen-rep2
0,Xkr4,32,40,31,33,18,17,24,26,0,29
1,Rp1,90,141,124,89,56,60,142,117,0,100
2,Sox17,5,9,3,2,1,3,2,2,0,4
3,Mrpl15,39,40,56,36,33,53,28,44,0,39
4,Lypla1,38,269,13,14,10,8,10,15,0,10
...,...,...,...,...,...,...,...,...,...,...,...
26209,Vmn2r122,0,0,0,0,0,0,0,0,0,0
26210,Vamp7,0,0,0,0,0,0,0,0,0,0
26211,Spry3,0,0,0,0,0,0,0,0,0,0
26212,Tmlhe,0,0,0,0,0,0,0,0,0,0


In [40]:
df_flt_rawcounts.Geneid.duplicated().sum()

0

In [41]:
# 过滤后的raw counts进入DESeq2处理
df_flt_rawcounts.to_csv(fc_file.replace('.txt', '_fix_feed_to_deseq2.csv'), index=None, sep=',')